Import the libraries for pre-processing

In [3]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf

In [4]:
tf.__version__

'2.3.0'

Data preprocessing

In [10]:
dataset = pd.read_csv('Churn_Modelling.csv')
X = dataset.iloc[:,3:-1].values
Y = dataset.iloc[:,-1].values
print(X)

[[619 'France' 'Female' ... 1 1 101348.88]
 [608 'Spain' 'Female' ... 0 1 112542.58]
 [502 'France' 'Female' ... 1 0 113931.57]
 ...
 [709 'France' 'Female' ... 0 1 42085.58]
 [772 'Germany' 'Male' ... 1 0 92888.52]
 [792 'France' 'Female' ... 1 0 38190.78]]


Impute missing data

In [ ]:
from sklearn.impute import SimpleImputer
imputer = SimpleImputer(missing_values = np.nan, strategy='mean')
imputer.fit(X[:,1:3])
X[:,1:3] = imputer.transform(X[:,1:3])


Encode Categorical Data

In [12]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
X[:,2] = le.fit_transform(X[:,2])
print(X)

[[619 'France' 0 ... 1 1 101348.88]
 [608 'Spain' 0 ... 0 1 112542.58]
 [502 'France' 0 ... 1 0 113931.57]
 ...
 [709 'France' 0 ... 0 1 42085.58]
 [772 'Germany' 1 ... 1 0 92888.52]
 [792 'France' 0 ... 1 0 38190.78]]


One hot encoding non-related data

In [14]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
X = np.array(ct.fit_transform(X))

In [15]:
print(X)

[[1.0 0.0 0.0 ... 1 1 101348.88]
 [0.0 0.0 1.0 ... 0 1 112542.58]
 [1.0 0.0 0.0 ... 1 0 113931.57]
 ...
 [1.0 0.0 0.0 ... 0 1 42085.58]
 [0.0 1.0 0.0 ... 1 0 92888.52]
 [1.0 0.0 0.0 ... 1 0 38190.78]]


Split the data into training and test data set

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,Y,test_size = 0.2, random_state=0) 

Feature Scaling

In [19]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.fit_transform(X_test)

**Build the ANN**

Initialize the ANN

In [20]:
ann = tf.keras.models.Sequential()


Add input layer and first hidden layer

In [21]:
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

Add second hidden layer

In [22]:
ann.add(tf.keras.layers.Dense(units=6,activation='relu'))

Add Output Layer

In [23]:
ann.add(tf.keras.layers.Dense(units=1,activation='sigmoid'))


**Training the ANN**

Compile the ANN

In [24]:
ann.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

Traning the ANN on the training set

In [25]:
ann.fit(X_train, y_train, batch_size = 32, epochs = 100)

Epoch 1/100
250/250 [==============================] - 0s 894us/step - loss: 0.5516 - accuracy: 0.7951
Epoch 2/100
250/250 [==============================] - 0s 953us/step - loss: 0.4961 - accuracy: 0.7960
Epoch 3/100
250/250 [==============================] - 0s 896us/step - loss: 0.4747 - accuracy: 0.7960
Epoch 4/100
250/250 [==============================] - 0s 907us/step - loss: 0.4602 - accuracy: 0.7960
Epoch 5/100
250/250 [==============================] - 0s 905us/step - loss: 0.4487 - accuracy: 0.7960
Epoch 6/100
250/250 [==============================] - 0s 888us/step - loss: 0.4392 - accuracy: 0.7960
Epoch 7/100
250/250 [==============================] - 0s 883us/step - loss: 0.4326 - accuracy: 0.7960
Epoch 8/100
250/250 [==============================] - 0s 869us/step - loss: 0.4274 - accuracy: 0.7972
Epoch 9/100
250/250 [==============================] - 0s 916us/step - loss: 0.4229 - accuracy: 0.8141
Epoch 10/100
250/250 [==============================] - 0s 927us/step - l

Inferences and Evaluation

In [31]:
print(ann.predict(sc.transform([[1, 0, 0, 600, 1, 40, 3, 6, 2, 1, 1, 50 ]])) > 0.5)

[[False]]


Predicting the test results

In [39]:
y_pred = ann.predict(X_test)
#print(y_pred)
y_pred = y_pred > 0.5
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

Confusion Matrix

In [40]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

[[1519   76]
 [ 197  208]]


0.8635